<a href="https://colab.research.google.com/github/ngtaylor/EE460J-Final-Project/blob/main/EE460J_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis using BERT


In [2]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text

     |████████████████████████████████| 4.9 MB 8.3 MB/s 


In [3]:
!pip install -q tf-models-official

     |████████████████████████████████| 1.8 MB 7.2 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 47.6 MB 100 kB/s 
     |████████████████████████████████| 1.2 MB 23.4 MB/s 
     |████████████████████████████████| 213 kB 57.1 MB/s 
     |████████████████████████████████| 352 kB 55.5 MB/s 
     |████████████████████████████████| 1.1 MB 56.5 MB/s 
     |████████████████████████████████| 99 kB 9.0 MB/s 
     |████████████████████████████████| 596 kB 49.2 MB/s 
     |████████████████████████████████| 90 kB 9.0 MB/s 


In [4]:
import os
import shutil
import zipfile

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import pandas as pd
import zipfile

from datetime import datetime

import matplotlib.pyplot as plt

In [5]:
!wget https://github.com/ngtaylor/EE460J-Final-Project/raw/main/data/training.1600000.processed.noemoticon.csv.zip

--2021-12-01 21:06:52--  https://github.com/ngtaylor/EE460J-Final-Project/raw/main/data/training.1600000.processed.noemoticon.csv.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ngtaylor/EE460J-Final-Project/main/data/training.1600000.processed.noemoticon.csv.zip [following]
--2021-12-01 21:06:52--  https://raw.githubusercontent.com/ngtaylor/EE460J-Final-Project/main/data/training.1600000.processed.noemoticon.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84855679 (81M) [application/zip]
Saving to: ‘training.1600000.processed.noemoticon.csv.zip’

training.1600000.pr 100%[=======

In [6]:
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv.zip', encoding='latin-1', names=["target", "id", "date", "flag", "user", "text"])

In [7]:
df.head()


,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [8]:
df.shape

(1600000, 6)

In [9]:
#Change positive target value from 4 to 1 and drop flag column since it is useless
df["target"] = df["target"].replace({4:1})
df = df.drop('flag', axis=1)
df.head()

,target,id,date,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."


In [10]:
#Shuffle values since they were ordered by target value
df = df.sample(frac = 1)
df.head()

,target,id,date,user,text
42699,0,1676217079,Fri May 01 21:24:45 PDT 2009,Tristan_xD,@sarahvip down syndrome chick bailed on me i ...
1025334,1,1883455024,Fri May 22 08:34:42 PDT 2009,curtishatter,get ready to go have some indian.. think chick...
1489722,1,2068743874,Sun Jun 07 14:37:12 PDT 2009,CompassWoman,@TwinHappyJen I tried but I am unable to follo...
1331363,1,2015986441,Wed Jun 03 05:56:41 PDT 2009,connster9700,@tkdsoulkat Thanks!
537933,0,2198927433,Tue Jun 16 16:46:18 PDT 2009,Ailyy,im sorry broke yu stapler ~moment of silence....
